# Uses GoogleMaps to find the GPS coordinates of the Venues

In [1]:
import pandas as pd
from pandas.io.json import json_normalize
from tqdm import tqdm
from datetime import datetime
import os

import googlemaps

In [2]:
#Need an api key to access Google Maps API.
with open('../../Api_keys/Google_maps.txt','r') as key:
    your_api_key = key.read().replace("'","")

In [3]:
data = pd.read_csv("./data/"+str(datetime.today())[:10]+"_GWD.csv")

In [4]:
data.head()

Unnamed: 0         city                              venue     dow  \
0           0     (Austin)   Black Star Co-Op Pub and Brewery  Sunday   
1           1     (Dallas)               Four Corners Brewing  Sunday   
2           2    (El Paso)  Glass Half Full (El Paso, Sunday)  Sunday   
3           3      (Wylie)                Good Neighbor Brews  Sunday   
4           4  (Grapevine)   Grapevine Craft Brewery (Sunday)  Sunday   

      time                                               urls  
0  8:00 PM  https://www.geekswhodrink.com/venue/19199768/t...  
1  5:00 PM  https://www.geekswhodrink.com/venue/771599801/...  
2  5:00 PM  https://www.geekswhodrink.com/venue/613308526/...  
3  5:00 PM  https://www.geekswhodrink.com/venue/576262892/...  
4  5:00 PM  https://www.geekswhodrink.com/venue/654369627/...

In [5]:
#cleaning the Data
data = data.drop(columns="Unnamed: 0")

austin = data[data['city'] == '(Austin)']

places = austin['venue'].values

In [6]:
gmaps = googlemaps.Client(key=your_api_key)

In [7]:
# Geocoding an address
austin_places = []
for place in tqdm(range(len(places))):
    geocode_result = gmaps.geocode(places[place] + " Austin, TX")
    austin_places.append(geocode_result)

100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [00:24<00:00,  1.76it/s]


### Creates a DataFrame form JSON 

In [8]:
google_maps_data = json_normalize([next(iter(place)) for place in austin_places])

In [9]:
austin = pd.read_csv('./data/'+str(datetime.today())[:10]+'_GWD_Austin.csv')

In [10]:
austin.head()

Unnamed: 0      city                                           venue  \
0           0  (Austin)                Black Star Co-Op Pub and Brewery   
1           1  (Austin)                                    The Cavalier   
2           2  (Austin)                                    The Highball   
3           3  (Austin)  400 Rabbits at Alamo Drafthouse Slaughter Lane   
4           4  (Austin)                 Austin Eastciders Collaboratory   

      dow     time                                               urls  
0  Sunday  8:00 PM  https://www.geekswhodrink.com/venue/19199768/t...  
1  Sunday  7:00 PM  https://www.geekswhodrink.com/venue/859909510/...  
2  Sunday  7:00 PM  https://www.geekswhodrink.com/venue/16729898/t...  
3  Monday  8:00 PM  https://www.geekswhodrink.com/venue/37034165/t...  
4  Monday  8:00 PM  https://www.geekswhodrink.com/venue/736622458/...

In [11]:
google_maps_data = austin.merge(google_maps_data,on=austin.index)

In [18]:
 google_maps_data = google_maps_data[['city','venue','dow','time','urls','formatted_address',
                  'geometry.location.lat','geometry.location.lng','types']]

In [19]:
#export the data
google_maps_data.to_csv('./data/'+str(datetime.today())[:10]+'_GWD_Austin_gmaps.csv',index=False)